In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import panel as pn
import requests

pn.extension()

# Load and prepare data
data = pd.read_csv('migration_details.csv')
data = data[data['region'] != 'Aggregates']

# Load GeoJSON for the world
geojson_url = "http://geojson.xyz/naturalearth-3.3.0/ne_50m_admin_0_countries.geojson"
response = requests.get(geojson_url)
geojson_data = response.json()

# Convert GeoJSON to a GeoDataFrame
world_geo = gpd.GeoDataFrame.from_features(geojson_data['features'])

# Metric options and years extraction
metric_translation = {
    'net_migration': 'Net migrants',
    'migration_percentage': 'Net migrants (% of population)',
}
years = sorted(data['year'].unique())

# Define function to create a Folium map
def create_folium_map(year, metric):
    m = folium.Map(location=[20, 0], zoom_start=2, tiles='cartodb positron')
    year_data = data[data['year'] == year][['iso3c', metric]].dropna()
    year_data = world_geo.merge(year_data, left_on='iso_a3', right_on='iso3c')
    
    folium.Choropleth(
        geo_data=geojson_data,
        data=year_data,
        columns=['iso_a3', metric],
        key_on='feature.properties.iso_a3',
        fill_color='Accent',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=metric_translation[metric]
    ).add_to(m)
    folium.GeoJson(
        data=year_data.__geo_interface__,
        style_function=lambda x: {'fillColor': '', 'color': 'rgba(0,0,0,0)', 'weight': 0, 'fillOpacity': 0},
        tooltip=folium.GeoJsonTooltip(
            fields=['iso_a3', metric],
            aliases=['Country: ', 'Population Density: '],
            localize=True
        )
    ).add_to(m)
    
    return m

# Panel widgets
year_slider = pn.widgets.IntSlider(name='Year', start=min(years), end=max(years), value=min(years), width=300)
metric_selector = pn.widgets.Select(name='Metric', options=list(metric_translation.keys()), value='net_migration', width=300)

# Interactive Panel object
@pn.depends(year=year_slider.param.value, metric=metric_selector.param.value)
def update_map(year, metric):
    folium_map = create_folium_map(year, metric)
    # Explicitly setting the width and height for the HTML pane
    return pn.pane.HTML(folium_map._repr_html_(), width=800, height=600)

# Layout
layout = pn.Column(
    pn.Row(year_slider, metric_selector),
    update_map,
    width=1000,  # Explicit width
    height=700   # Explicit height, adjusted to ensure proper display
)

# Save layout as HTML with embedded resources
from bokeh.resources import INLINE
layout.save('dashboard.html', embed=True, resources=INLINE)